In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Darknet.csv to Darknet.csv


In [ ]:
import os
os.listdir()


['.config', 'sample_data']

In [ ]:
!pip install deap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from deap import base, creator, tools, algorithms


In [ ]:
data = pd.read_csv("Darknet.csv")

print("Dataset Shape:", data.shape)
data.head()


Dataset Shape: (158616, 85)


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label.1
0,10.152.152.11-216.58.220.99-57158-443-6,10.152.152.11,57158,216.58.220.99,443,6,24/07/2015 04:09:48 PM,229,1,1,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
1,10.152.152.11-216.58.220.99-57159-443-6,10.152.152.11,57159,216.58.220.99,443,6,24/07/2015 04:09:48 PM,407,1,1,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
2,10.152.152.11-216.58.220.99-57160-443-6,10.152.152.11,57160,216.58.220.99,443,6,24/07/2015 04:09:48 PM,431,1,1,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
3,10.152.152.11-74.125.136.120-49134-443-6,10.152.152.11,49134,74.125.136.120,443,6,24/07/2015 04:09:48 PM,359,1,1,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
4,10.152.152.11-173.194.65.127-34697-19305-6,10.152.152.11,34697,173.194.65.127,19305,6,24/07/2015 04:09:45 PM,10778451,591,400,...,0,0,0,0,1.437765e+15,3.117718e+06,1.437765e+15,1.437765e+15,Non-Tor,AUDIO-STREAMING


In [ ]:
TARGET_COLUMN = "Label"

X = data.drop(columns=[TARGET_COLUMN])
y = data[TARGET_COLUMN]


In [ ]:
for col in X.select_dtypes(include="object").columns:
    X[col] = LabelEncoder().fit_transform(X[col])

if y.dtype == "object":
    y = LabelEncoder().fit_transform(y)


##  Feature Scaling


In [ ]:
import numpy as np

# Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Check total missing values
print("Total missing values before imputation:", X.isna().sum().sum())

# Impute missing values with column mean
X = X.fillna(X.mean())

# Now apply scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

feature_names = X.columns
num_features = X_scaled.shape[1]

print("Scaling completed: Data cleaned and scaled successfully")


Total missing values before imputation: 100
Scaling completed: Data cleaned and scaled successfully


##  Genetic Algorithm Setup


In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.attr_bool,
    n=num_features
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


## Define Fitness Function (Random Forest Accuracy)


In [ ]:
def fitness_function(individual):
    selected = [i for i, bit in enumerate(individual) if bit == 1]

    if len(selected) == 0:
        return 0.0,

    X_sel = X_scaled[:, selected]

    model = RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    )

    accuracy = cross_val_score(
        model, X_sel, y, cv=5, scoring="accuracy"
    ).mean()

    return accuracy,


##  Genetic Algorithm Operators


In [ ]:
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


##  Run Genetic Algorithm for Feature Selection


In [ ]:
population = toolbox.population(n=30)
generations = 15

algorithms.eaSimple(
    population,
    toolbox,
    cxpb=0.7,
    mutpb=0.2,
    ngen=generations,
    verbose=True
)

best_individual = tools.selBest(population, 1)[0]
selected_indices = [i for i, bit in enumerate(best_individual) if bit == 1]

print("Selected Features:")
feature_names[selected_indices]


KeyboardInterrupt: 

##  Train Final Random Forest Model


In [ ]:
X_final = X_scaled[:, selected_indices]

X_train, X_test, y_train, y_test = train_test_split(
    X_final,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)


## Model Evaluation


In [ ]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


##  Save Selected Features


In [ ]:
pd.DataFrame(
    feature_names[selected_indices],
    columns=["Selected Features"]
).to_csv("selected_features.csv", index=False)

files.download("selected_features.csv")
